### 马尔可夫过程

#### 随机过程

在随机过程中，随机现象在某时刻 $t$ 的取值是一个向量随机变量，用 $S_t$ 表示，所有可能的状态组成状态集合 $S$。

将已知历史信息 $(S_1,\dots,S_t)$ 时下一个时刻状态为 $S_{t+1}$ 的概率表示成 $P(S_{t+1}|S_1,\dots,S_t)$。

#### 马尔可夫性质

当且仅当某时刻的状态只取决于上一时刻的状态时，一个随机过程被称为具有马尔可夫性质，即 $P(S_{t+1}|S_t)=P(S_{t+1}|S_1,\dots,S_t)$。

#### 马尔可夫过程

马尔可夫过程指具有马尔可夫性质的随机过程，也称作马尔科夫链，通常用 $\langle S,P\rangle$ 来描述，其中 $S$ 是有限数量的状态集合，$P$ 是状态转移矩阵，定义了所有状态对之间的转移概率，即

$$
P=
\begin{bmatrix}
P(s_1\mid s_1)&\dots&P(s_n\mid s_1)\\
\vdots&\ddots&\vdots\\
P(s_1\mid s_n)&\dots&P(s_n\mid s_n)
\end{bmatrix}
$$

给定一个马尔可夫过程，可以从某个状态出发，根据状态转移矩阵生成一个状态序列，这个步骤也被称为采样。

### 马尔可夫奖励过程

一个马尔可夫奖励过程由 $\langle S,P,r,\gamma \rangle$ 组成。$r$ 是奖励函数，某个状态 $s$ 的奖励 $r(s)$ 指转移到该状态时可以获得奖励的期望。$\gamma$ 是折扣因子，取值范围是 $[0,1)$，当 $\gamma$ 接近 $0$ 时更考虑短期奖励，否则更考虑长期奖励。

#### 回报

在一个马尔科夫奖励过程中，从第 $t$ 时刻状态 $S_t$ 开始，直到终止状态时，所有奖励的衰减之和称为回报 $G_t$，公式如下：

$$
G_t=R_t+\gamma R_{t+1}+\gamma^2R_{t+2}+\cdots=\sum_{k=0}^{\infty}{\gamma^kR_{t+k}}
$$

尝试用代码实现一个马尔科夫奖励过程。

In [1]:
import numpy as np

np.random.seed(0)
P = np.array([
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]) # 状态转移矩阵
rewards = [-1, -2, -2, 10, 1, 0] # 奖励函数
gamma = 0.5 # 折扣因子

def compute_return (chain, gamma, start_index=0):
    G = 0
    for i in reversed(range(start_index, len(chain))):
        G = G * gamma + rewards[chain[i] - 1]
    return G

chain = [1, 2, 3, 6]
G = compute_return(chain, gamma)
print("本序列的回报为：", G)


本序列的回报为： -2.5


#### 价值函数

一个状态的期望回报称为这个状态的价值，所有状态的价值就组成了价值函数， 即 $V(s)=\mathbb{E}[G_t|S_t=s]$，展开为

$$
\begin{aligned}
V(s)&=\mathbb{E}[G_t|S_t=s]\\
&=\mathbb{E}[R_t+\gamma R_{t+1}+\gamma^2R_{t+2}+\dots|S_t=s]\\
&=\mathbb{E}[R_t+\gamma (R_{t+1}+\gamma R_{t+2}+\dots)|S_t=s]\\
&=\mathbb{E}[R_t+\gamma G_{t+1}|S_t=s]\\
&=\mathbb{E}[R_t+\gamma V(S_{t+1})|S_t=s]\\
\end{aligned}
$$

左半部是奖励函数的输出，即 $\mathbb{E}[R_t|S_t=s]=r(s)$，而右半部可以由状态 $s$ 出发的转移概率得到，即

$$
V(s)=r(x)+\gamma\sum_{s'\in S}{p(s'|s)V(s')}
$$

该式子即为贝尔曼方程。

设一个马尔可夫过程有 $n$ 个状态，即 $S=\{s_1,s_2,\dots,s_n\}$，概率转移矩阵为 $P$，将所有状态的价值表示成一个列向量 $V=[V(s_1),V(s_2),\dots,V(s_n)]^T$，将所有奖励函数写成一个列向量 $R=[r(s_1),r(s_2),\dots,r(s_n)]^T$，则可以得到贝尔曼方程的矩阵形式为

$$
V=R+\gamma PV
$$

其解析解为 $V=(I-\gamma P)^{-1}R$，计算改解析解的复杂度为 $O(n^3)$。

下面用代码实现。

In [2]:
# 利用贝尔曼方程的矩阵形式计算解析式
def compute (P, rewards, gamma, states_num):
    rewards = np.array(rewards).reshape((-1, 1)) # 转换成列向量形式
    value = np.dot(np.linalg.inv(np.eye(states_num, states_num) - gamma * P), rewards)
    return value

V = compute(P, rewards, gamma, 6)
print("MRP中每个状态的价值分别为：\n", V)

MRP中每个状态的价值分别为：
 [[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


### 马尔可夫决策过程

在马尔可夫奖励过程中加入智能体的动作后，就变成了马尔可夫决策过程。一个马尔可夫决策过程由 $\langle S,A,P,r,\gamma \rangle$ 组成，其中 $S$ 是状态的集合，$A$ 是动作的集合，$\gamma 是折扣因子$，$r(s,a)$ 是奖励函数，$P(s'|s,a)$ 是状态转移函数。

在马尔科夫决策过程中，智能体根据当前状态 $S_t$ 选择动作 $A_t$，MDP 根据奖励函数和状态转移函数得到 $S_{t+1}$ 和 $R_t$ 并反馈给智能体，智能体的目标是最大化累计奖励，并从 $A$ 中选择一个动作，称为决策。

#### 策略

智能体的策略用 $\pi$ 表示，即 $\pi(a|s)=P(A_t=a|S_t=s)$，表示在状态 $s$ 下采取动作 $a$ 的概率。当一个策略是确定性策略时，它在每个状态只有一个确定性的动作，否则是随机性策略。

#### 状态价值函数

定义为从状态 $s$ 出发遵循策略 $\pi$ 能获得的期望回报，即

$$
V^{\pi}(s)=\mathbb{E}_{\pi}[G_t|S_t=s]
$$

#### 动作价值函数

用 $Q^{\pi}(s,a)$ 表示遵循策略 $\pi$ 时，对当前状态 $s$ 执行动作 $a$ 得到的期望回报，即

$$
Q^{\pi}(s,a)=\mathbb{E}_{\pi}[G_t|S_t=s,A_t=a]
$$

状态价值函数和动作价值函数的关系

$$
V^{\pi}(s)=\sum_{a\in A} \pi(a|s)Q^{\pi}(s,a)
$$

使用策略 $\pi$ 时，状态 $s$ 下采取动作 $a$ 的价值等于即时奖励加上经过衰减后的所有可能的下一状态的状态转移概率与对应价值的乘积，即

$$
Q^{\pi}(s,a)=r(s,a)+\gamma\sum_{s'\in S}P(s'|s,a)V^{\pi}(s')
$$

#### 贝尔曼期望方程

$$
\begin{aligned}
V^{\pi}(s)&=\mathbb{E}_{\pi}[R_t+\gamma V^{\pi}(S_{t+1})|S_t=s]\\
&=\sum_{a\in A}\pi(a|s)\left (r(s,a)+\gamma\sum_{s'\in S}p(s'|s,a)V^{\pi}(s')\right )
\end{aligned}
$$

$$
\begin{aligned}
Q^{\pi}(s,a)&=\mathbb{E}_{\pi}[R_t+\gamma Q^{\pi}(S_{t+1},A_{t+1})|S_t=s,A_t=a]\\
&=r(s,a)+\gamma\sum_{s'\in S}p(s'|s,a)\sum_{a'\in A}\pi(a'|s')Q^{\pi}(s',a')
\end{aligned}
$$

下面用代码实现一个马尔可夫决策过程。

In [3]:
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态集合
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作集合
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
} # 状态转移函数
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
} # 奖励函数
gamma = 0.5 # 折扣因子
MDP = (S, A, P, R, gamma)
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
} # 策略1,随机策略
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
} # 策略2

# 把输入的两个字符串通过“-”连接
def join(str1, str2):
    return str1 + '-' + str2

考虑计算该 MDP 下，一个策略 $\pi$ 的状态价值函数，可以将策略的动作选择进行边缘化，就可以得到没有动作的 MRP。

对于一个状态，根据策略所有动作的概率进行加权，得到一个 MRP 在该状态下的奖励，即

$$
r'(s)=\sum_{a\in A}\pi(a|s)r(s,a)
$$

同理可以得到一个 MRP 从状态 $s$ 转移到状态 $s'$ 的概率，即

$$
P'(s'|s)=\sum_{a\in A}\pi(a|s)P(s'|s,a)
$$

于是可以得到一个 MRP: $\langle S,P',r',\gamma \rangle$，我们可以用 MRP 中计算价值函数的解析解来计算这个 MDP 中该策略的状态价值函数。

下面利用该方法计算使用随机策略时的状态价值函数。

In [4]:
gamma = 0.5
P_from_mdp_to_mrp = np.array([
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]) # 转化后的状态转移矩阵
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0]

V = compute(P_from_mdp_to_mrp, R_from_mdp_to_mrp, gamma, 5)
print("MDP中每个状态价值分别为\n", V)

MDP中每个状态价值分别为
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


知道了状态价值函数 $V^{\pi}(s)$ 后，动作价值函数 $Q^{\pi}(s,a)$ 也可以根据定义一并算出。

### 蒙特卡洛方法

蒙特卡洛方法是一种基于概率统计的数值计算方法，通常使用重复随机抽样，然后运用概率统计方法来从抽样结果中归纳出我们想求的目标的数值估计。

那么运用蒙特卡洛方法计算状态价值时，为每一个状态维护一个计数器和总回报，过程如下：

（1）使用策略 $\pi$ 采样若干条序列。

（2）对每一条序列的每一时间步 $t$ 的状态 $s$ 进行以下操作：

- 更新状态 $s$ 的计数器 $N(s)\gets N(s)+1$；
- 更新状态 $s$ 的总回报 $M(s)\gets M(s)+G_t$

（3）每一个状态的价值被估计为回报的平均值 $V(s)=M(s)/N(s)$。

同时还有一种增量更新的方法：

- $N(s)\gets N(s)+1$
- $V(s)\gets V(s)+\frac{1}{N(s)}(G-V(s))$

下面用代码实现。

In [5]:
def sample (MDP, Pi, timestep_max, number):
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = []
        timestep = 0
        s = S[np.random.randint(4)] # 随机选择一个非 s5 的状态

        while (s != "s5" and timestep <= timestep_max):
            timestep += 1
            rand, temp = np.random.rand(), 0
            # 选择动作
            for a_opt in A:
                temp += Pi.get(join(s, a_opt), 0)
                if (temp > rand):
                    a = a_opt
                    r = R.get(join(s, a), 0)
                    break
            rand, temp = np.random.rand(), 0
            # 选择下一状态
            for s_opt in S:
                temp += P.get(join(join(s, a), s_opt), 0)
                if (temp >= rand):
                    s_next = s_opt
                    break
            episode.append((s, a, r, s_next))
            s = s_next # 从下一状态继续循环
        episodes.append(episode)
    return episodes

episodes = sample(MDP, Pi_1, 20, 5)
for i in range(5):
    print("第{}条序列：".format(i + 1), episodes[i])

第1条序列： [('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s5', 0, 's5')]
第2条序列： [('s4', '概率前往', 1, 's4'), ('s4', '前往s5', 10, 's5')]
第3条序列： [('s4', '前往s5', 10, 's5')]
第4条序列： [('s2', '前往s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]
第5条序列： [('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]


In [6]:
# 对采样序列计算所有状态的价值
def MC (episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        # 从后往前计算
        for i in range(len(episode) - 1, -1, -1):
            (s, a, r, s_next) = episode[i]
            G = r + gamma * G
            N[s] += 1
            V[s] += (G - V[s]) / N[s]

episodes = sample(MDP, Pi_1, timestep_max=20, number=10000)
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
MC(episodes, V, N, gamma=0.5)
print("使用蒙特卡洛方法计算 MDP 的状态价值为\n", V)

使用蒙特卡洛方法计算 MDP 的状态价值为
 {'s1': -1.2206481669904494, 's2': -1.6886433026750503, 's3': 0.49346512992377956, 's4': 6.037788973046783, 's5': 0}


### 占用度量

定义 MDP 的初始状态分布为 $\nu_0(s)$，用 $P_t^{\pi}(s)$ 表示采取策略 $\pi$ 使智能体在 $t$ 时刻的状态为 $s$ 概率，显然有 $P_0^{\pi}(s)=\nu_0(s)$。然后可以定义一个策略的状态访问分布，即

$$
\nu^{\pi}(s)=(1-\gamma)\sum_{t=0}^{\infty}\gamma^tP_t^{\pi}(s)
$$

状态访问概率有如下性质

$$
\nu^{\pi}(s')=(1-\gamma)\nu_0(s')+\gamma\int P(s'|s,a)\pi(a|s)\nu^{\pi}(s)dsda
$$

此外还可以定义策略的占用度量，它表示动作状态对 $(s,a)$ 被访问到的概率，即

$$
\rho^{\pi}(s,a)=(1-\gamma)\sum_{t=0}^{\infty}\gamma^tP_t^{\pi}(s)\pi(a|s)=\nu^{\pi}(s)\pi(a|s)
$$

进一步可以得到下面两个定理。

**定理1：** $$\rho^{\pi_1}=\rho^{\pi_2}\Longleftrightarrow \pi_1=\pi_2$$

**定理2：** 给定一合法占用度量 $\rho$，可生成该占用度量的唯一策略为 $$\pi_{\rho}=\frac{\rho(s,a)}{\sum_{a'}\rho(s,a')}$$

下面用代码来近似估计占用度量。

In [7]:
# 计算状态动作对 (s, a) 出现的频率来估算策略的占用度量
def occupancy (episodes, s, a, timestep_max, gamma):
    rho = 0
    total_times = np.zeros(timestep_max)
    occur_times = np.zeros(timestep_max)
    
    for episode in episodes:
        for i in range(len(episode)):
            (s_opt, a_opt, r, s_next) = episode[i]
            total_times[i] += 1
            if (s == s_opt and a == a_opt):
                occur_times[i] += 1
    for i in range(timestep_max):
        if (total_times[i] != 0):
            rho += (gamma ** i) * occur_times[i] / total_times[i]
    return (1 - gamma) * rho

episodes_1 = sample(MDP, Pi_1, 1000, 1000)
episodes_2 = sample(MDP, Pi_2, 1000, 1000)
rho_1 = occupancy(episodes_1, "s4", "概率前往", 1000, 0.5)
rho_2 = occupancy(episodes_2, "s4", "概率前往", 1000, 0.5)
print(rho_1, rho_2)

0.117927768265181 0.23436384017172737


### 最优策略

定义策略之间的偏序关系：当且仅当对于任意的状态 $s$ 都有 $V^{\pi}(s)\geq V^{\pi'}(s)$，记为 $\pi >\pi'$。

当一个策略不差于其他所有策略，则称其为最优策略，表示为 $\pi^*(s)$。

最优状态价值函数为 $$V^*(s)=\max_{\pi}V^{\pi}(s),\forall s\in S$$

最优动作价值函数为 $$Q^*(s,a)=\max_{\pi}Q^{\pi}(s,a),\forall s\in S,a\in A$$

为了使 $Q^{\pi}(s,a)$ 最大，需要总是执行最优策略，所以最优状态价值函数和最优动作价值函数的关系为 $$Q^*(s,a)=r(s,a)+\gamma\sum_{s'\in S}P(s'|s,a)V^*(s')$$

另一方面，最有状态价值函数总是选择最优动作价值函数最大的动作的状态价值，即 $$V^*(s)=\max_{a\in A}Q^*(s,a)$$

#### 贝尔曼最优方程

$$V^*(s)=\max_{a\in A}\{r(s,a)+\gamma\sum_{s'\in S}p(s'|s,a)V^*(s')\}$$

$$Q^*(s,a)=r(s,a)+\gamma\sum_{s'\in S}p(s'|s,a)\max_{a'\in A}Q^*(s',a')$$